In [127]:
import json, requests
from bs4 import BeautifulSoup
import sys
import time
import urllib.parse as urlparse
from urllib.parse import urlencode

In [128]:
u = 'http://api.nytimes.com/svc/search/v2/articlesearch.json'
datePair = [('20180326', '20180327'), ('20180328', '20180329'), ('20180330', '20180331'), ('20180401', '20180402')]

In [129]:
parameter = {
    'api-key': "your key",
    'q': "big data",
    'begin_date': "20180326",
    'end_date': "20180327",
    'page':0
}

In [130]:
def getUrl(u,parameter):
    url_parts = list(urlparse.urlparse(u))
    query = dict(urlparse.parse_qsl(url_parts[4]))
    query.update(parameter)
    url_parts[4] = urlencode(query)
    return urlparse.urlunparse(url_parts)

In [131]:
def getHtmltxt(docid,jsdata):
    webpage = requests.get(jsdata["response"]["docs"][docid]["web_url"])
    soup = BeautifulSoup(webpage.text, 'html.parser')
    p = soup.find_all('p')
    soup = BeautifulSoup(' '.join(str(i) for i in p), 'html.parser')
    return soup.getText()

In [132]:
page = 0

In [133]:
def tryGetData(url):
    resp = requests.get(url)
    data = json.loads(resp.text)
    try:
        hits = data['response']['meta']['hits']
    except KeyError:
        print(data)
        time.sleep(10)
        data,hits = tryGetData(url)
    return data,hits

In [134]:
for dPair in datePair:# each 2 days' data
    parameter['begin_date'],parameter['end_date'] = dPair
    parameter['page'] = page
    url = getUrl(u,parameter)
    data,hits = tryGetData(url)
    docid = 0
    print(f'got {hits} results from this date')
    print(url)
    time.sleep(1)
    while(hits>=1):
        f= open(f"NewsData/{dPair[0]}-{dPair[1]}.txt","a",encoding='utf-8')
        try:
            ids = len(data["response"]["docs"])
        except KeyError:
            print(data)
            time.sleep(10)
            continue
        for docid in range(ids):
            f.writelines(getHtmltxt(docid,data) + "\n")
            hits -= 1
            if(hits<1):
                break
        page += 1
        if(page>200):
            break
        parameter['page'] = page
        url = getUrl(u,parameter)
        resp = requests.get(url)
        if(hits<1):
            break
        data = json.loads(resp.text)
        f.close()
        time.sleep(1)
    page = 0         

got 175 results from this date
http://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=46c1eeb270c94547869a041d823b86ac&q=big+data&begin_date=20180326&end_date=20180327&page=0
{'message': 'API rate limit exceeded'}
got 220 results from this date
http://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=46c1eeb270c94547869a041d823b86ac&q=big+data&begin_date=20180328&end_date=20180329&page=0
{'message': 'API rate limit exceeded'}
got 152 results from this date
http://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=46c1eeb270c94547869a041d823b86ac&q=big+data&begin_date=20180330&end_date=20180331&page=0
{'message': 'API rate limit exceeded'}
got 92 results from this date
http://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=46c1eeb270c94547869a041d823b86ac&q=big+data&begin_date=20180401&end_date=20180402&page=0


In [126]:
f.close()